In [1]:
import torch

# Example tensor
tensor = torch.randn(2, 3)  # Create a random tensor

# Save tensor
torch.save(tensor, 'example_tensor.pt')

# Example model
model = torch.nn.Linear(2, 3)  # A simple linear model
torch.save(model.state_dict(), 'model_weights.pt')  # Save model's state dictionary
